In [2]:
import customtkinter as ctk
import importlib.util
import json
import os
import pkgutil
import re
import sys

# List of standard libraries
standard_libs = {name.lower() for name in sys.builtin_module_names}

def parse_notebook(file_path):
    with open(file_path, 'r') as file:
        notebook = json.load(file)
    
    code_lines = []
    for cell in notebook['cells']:
        if cell['cell_type'] == 'code':
            code_lines.extend(cell['source'])
    
    return code_lines

def extract_libraries(lines):
    required_libraries = set()
    for line in lines:
        match_import = re.match(r'^\s*import\s+(\w+)', line)
        match_from = re.match(r'^\s*from\s+(\w+)', line)
        if match_import:
            required_libraries.add(match_import.group(1).lower())
        elif match_from:
            required_libraries.add(match_from.group(1).lower())
    
    return required_libraries

def check_installed_libraries(file_path):
    if file_path.endswith('.ipynb'):
        lines = parse_notebook(file_path)
    else:
        with open(file_path, 'r') as file:
            lines = file.readlines()
    
    required_libraries = extract_libraries(lines)
    
    external_libraries = {lib for lib in required_libraries if lib not in standard_libs}
    installed_libraries = {lib for lib in external_libraries if importlib.util.find_spec(lib) is not None}
    missing_libraries = external_libraries - installed_libraries
    
    print(f"Required libraries: {required_libraries}")
    print(f"External libraries: {external_libraries}")
    print(f"Installed libraries: {installed_libraries}")
    print(f"Missing libraries: {missing_libraries}")
    
    return external_libraries, missing_libraries

def select_file():
    app = ctk.CTk()
    app.withdraw()  # Hide the root window

    file_path = ctk.filedialog.askopenfilename(filetypes=[("Python files", "*.py"), ("Jupyter notebooks", "*.ipynb")])
    if file_path:
        print(f"Selected file: {file_path}")
        external_libraries, missing_libraries = check_installed_libraries(file_path)
        if missing_libraries:
            print(f"The following libraries are missing: {missing_libraries}")
        else:
            print("All required external libraries are installed.")
        print(f"The required external libraries are: {external_libraries}")

    app.destroy()

if __name__ == "__main__":
    select_file()
